# 🧪 Interactive ADM1 Workshop: UK Feedstocks

Welcome! This notebook allows you to explore the **ADM1 model** interactively with common UK AD feedstocks.

<details>
<summary>Step 1: Feedstock Database</summary>

- Four UK AD feedstocks: maize silage, grass silage, food waste, cattle slurry
- Characteristics: carbohydrates, proteins, lipids, VFAs, TAN, moisture, cations, anions
- Database auto-saved as `Feedstock_Characteristics_Database.xlsx`
</details>

<details>
<summary>Step 2: Feeding Plan</summary>

- Set fractions of each feedstock (auto-normalised to sum = 1)
- Set HRT, Reactor Volume (V), Flow (Q), Temperature (°C)
- Feeding plan saved as `Fixed_Feeding.xlsx`
</details>

<details>
<summary>Step 3: Run ADM1 Simulator</summary>

- Calculates Methane, Biogas, pH, FOS/TAC, Gas Pressure
- Time-series plots generated
- Summary printed at the end
</details>

<details>
<summary>Step 4: Interactive Sliders</summary>

- Adjust feed fractions and process parameters
- Observe trends in outputs
</details>

<details>
<summary>Step 5: Export Results</summary>

- Download CSV of simulation results using `export_results()`
</details>

In [ ]:
!pip install matplotlib ipywidgets pandas openpyxl

: 

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Import ADM1 simulator from ADM1.py
from ADM1 import ADM1Simulator

In [107]:
# Step 1: Feedstock Database is now hardcoded in ADM1.py, no need to generate or save Excel.

In [108]:
# Step 2: Feeding Plan - define feedstock ratios and process parameters
def get_feedstock_ratios(maize_silage, grass_silage, food_waste, cattle_slurry):
    ratios = {
        "Maize Silage": maize_silage,
        "Grass Silage": grass_silage,
        "Food Waste": food_waste,
        "Cattle Slurry": cattle_slurry
    }
    total = sum(ratios.values())
    if total > 0:
        ratios = {k: v/total for k, v in ratios.items()}
    return ratios

In [109]:
# Step 3: Run ADM1 Simulator
def run_adm1(maize_silage, grass_silage, food_waste, cattle_slurry, HRT, V, Q, T):
    ratios = get_feedstock_ratios(maize_silage, grass_silage, food_waste, cattle_slurry)
    days = int(HRT)
    simulator = ADM1Simulator(ratios, days=days, Q=Q, V=V, T=T)
    simulator.run()
    output_data = simulator.get_results()
    if output_data.empty:
        print("⚠️ Simulation returned no data.")
        return
    # Patch: If 'V' is not present, add it for compatibility
    if 'V' not in output_data.columns:
        output_data.insert(len(output_data.columns), 'V', V)
    summary = {
        "pH": output_data["pH"].iloc[-1],
        "FOS": output_data["FOS"].iloc[-1],
        "TAC": output_data["TAC"].iloc[-1],
        "FOS/TAC": output_data["FOS/TAC"].iloc[-1],
        "Methane Flow Rate": output_data["q_ch4"].iloc[-1],
        "Biogas": output_data["q_gas"].iloc[-1],
        "Gas Pressure": output_data["p_gas"].iloc[-1]
    }
    fig, ax = plt.subplots(3,1, figsize=(8,10), sharex=True)
    ax[0].plot(output_data["time"], output_data["q_ch4"], label="Methane Flow")
    ax[0].set_ylabel("Methane (m³/d)")
    ax[0].grid(True)
    ax[1].plot(output_data["time"], output_data["pH"], color="green", label="pH")
    ax[1].set_ylabel("pH")
    ax[1].grid(True)
    ax[2].plot(output_data["time"], output_data["FOS"], color="red", label="FOS")
    ax[2].plot(output_data["time"], output_data["TAC"], color="blue", label="TAC")
    ax[2].set_ylabel("FOS / TAC")
    ax[2].legend()
    ax[2].set_xlabel("Time (days)")
    ax[2].grid(True)
    plt.suptitle(f"Feed fractions → maize:{maize_silage}, grass:{grass_silage}, food:{food_waste}, slurry:{cattle_slurry}", fontsize=12)
    plt.show()
    print("📊 Simulation summary:")
    for k,v in summary.items():
        print(f"- {k}: {v:.2f}")

In [ ]:
# Step 4: Interactive Sliders
interact(
    run_adm1,
    maize_silage=FloatSlider(min=0, max=1, step=0.1, value=0.5, description="Maize Silage"),
    grass_silage=FloatSlider(min=0, max=1, step=0.1, value=0.3, description="Grass Silage"),
    food_waste=FloatSlider(min=0, max=1, step=0.1, value=0.1, description="Food Waste"),
    cattle_slurry=FloatSlider(min=0, max=1, step=0.1, value=0.1, description="Cattle Slurry"),
    HRT=FloatSlider(min=10, max=80, step=2, value=48, description="HRT (days)"),
    V=FloatSlider(min=1000, max=10000, step=500, value=6520, description="V (m³)"),
    Q=FloatSlider(min=50, max=500, step=10, value=136.63, description="Q (m³/d)"),
    T=FloatSlider(min=15, max=60, step=1, value=45, description="T (°C)"),
);

In [113]:
# Step 5: Export Results
def export_results(maize_silage=0.5, grass_silage=0.3, food_waste=0.1, cattle_slurry=0.1,
                   HRT=48, V=6520, Q=136.63, T=45):
    ratios = get_feedstock_ratios(maize_silage, grass_silage, food_waste, cattle_slurry)
    days = int(HRT)
    simulator = ADM1Simulator(ratios, days=days, Q=Q, V=V, T=T)
    simulator.run()
    output_data = simulator.get_results()
    filename = "ADM1_results.csv"
    output_data.to_csv(filename, index=False)
    print(f"✅ CSV exported: {filename}")